In [389]:
import dlib
import cv2
import os
import pandas as pd
import numpy as np

In [390]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [391]:
def convertToGray(img): # function that convert image to grayscale.
	return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [392]:
def get_landmarks(imag):
	landmarks = []
	detections = detector(imag, 1)
    
	for k,d in enumerate(detections): #For all detected face instances individually
		shape = predictor(imag, d) #Draw Facial Landmarks with the predictor class
		xlist = []
		ylist = []
		for i in range(0,68): #Store X and Y coordinates in two lists
			xlist.append(float(shape.part(i).x))
			ylist.append(float(shape.part(i).y))

		for x, y in zip(xlist, ylist): #Store all landmarks in one list in the format x1,y1,x2,y2,etc.
			landmarks.append(x)
			landmarks.append(y)
            
	if len(detections) > 0:
		return landmarks
	else: #If no faces are detected, return error message to other function to handle
		landmarks = "error"
		return landmarks

In [393]:
def extractFrames(pathIn, pathOut):
	df = pd.DataFrame()
	os.mkdir(pathOut)

	land_array = []
	col = []
        
	j = 1
	k1 = 1
	k2 = 1
	for j in range(1, 137):
		if (j%2) == 0:
			k1 = j/2
			col.append("land_%d_y" % k1)
		else:
			k2 = (j/2) + 0.5 # Ao pegar o resto, tiro ele do valor de j para colocar na land_x
			col.append("land_%d_x" % k2)
		j += 1
	print(col)
    
	cap = cv2.VideoCapture(pathIn)
	count = 0
	bias = 50
	while (cap.isOpened() and count<10): # o contador<10 serve para tirar apenas os 10 primeiros frames do video

		# Capture frame-by-frame
		ret, frame = cap.read()

		if ret == True:
			print('Read %d frame: ' % count, ret)

			frame = convertToGray(frame) # convert image to gray
			# uses the classifier HaarCascade to extract the faces.
			haar_face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
			faces = haar_face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5) # detecta quantas faces ha na imagem

			for (x,y,w,h) in faces: # Para cada face faz um corte
				crop_img = frame[y-bias: y+h+bias, x-bias: x+w+bias]

			landmark = get_landmarks(crop_img) # Aqui recebo lista com landmarks.
			#print(type(landmark)) # Podemos ver com isso que a variavel retornada pela funcao get_landmarks eh do tipo list.
			print(len(landmark)) # Tamanho da lista
			#array = np.array([landmark])
            #pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)]
			land_array = pd.DataFrame([landmark], index = ["frame_%d" % count], columns = col)
			print(land_array)
			df = df.append(land_array)

			clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
			clahe_image = clahe.apply(crop_img) # Aplicação do Clahe a imagem ja em cinza
			detections = detector(clahe_image, 1)#Detect the faces in the image
			for k,d in enumerate(detections): #For each detected face
            			shape = predictor(clahe_image, d) #Get coordinates
            			for i in range(1,68): #There are 68 landmark points on each face
                     			#For each point, draw a red circle with thickness2 on the original frame
                    			cv2.circle(crop_img, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2)

			cv2.imwrite(os.path.join(pathOut, "frame{:d}.jpg".format(count)), crop_img)  # save crop_img as JPEG file

			count += 1
		else:
			break

	print(df)
	#print(df.shape)
	# When everything done, release the capture
	cap.release()
	#cv2.destroyAllWindows()


In [394]:
def createDataFrame():

	df = pd.DataFrame()
	print(df)

In [395]:
def main():
	extractFrames('Em1_Fala1_CarolinaHolly.mp4', 'A1')

In [396]:
if __name__=="__main__":
	main()

['land_1_x', 'land_1_y', 'land_2_x', 'land_2_y', 'land_3_x', 'land_3_y', 'land_4_x', 'land_4_y', 'land_5_x', 'land_5_y', 'land_6_x', 'land_6_y', 'land_7_x', 'land_7_y', 'land_8_x', 'land_8_y', 'land_9_x', 'land_9_y', 'land_10_x', 'land_10_y', 'land_11_x', 'land_11_y', 'land_12_x', 'land_12_y', 'land_13_x', 'land_13_y', 'land_14_x', 'land_14_y', 'land_15_x', 'land_15_y', 'land_16_x', 'land_16_y', 'land_17_x', 'land_17_y', 'land_18_x', 'land_18_y', 'land_19_x', 'land_19_y', 'land_20_x', 'land_20_y', 'land_21_x', 'land_21_y', 'land_22_x', 'land_22_y', 'land_23_x', 'land_23_y', 'land_24_x', 'land_24_y', 'land_25_x', 'land_25_y', 'land_26_x', 'land_26_y', 'land_27_x', 'land_27_y', 'land_28_x', 'land_28_y', 'land_29_x', 'land_29_y', 'land_30_x', 'land_30_y', 'land_31_x', 'land_31_y', 'land_32_x', 'land_32_y', 'land_33_x', 'land_33_y', 'land_34_x', 'land_34_y', 'land_35_x', 'land_35_y', 'land_36_x', 'land_36_y', 'land_37_x', 'land_37_y', 'land_38_x', 'land_38_y', 'land_39_x', 'land_39_y', 'la